## Find the best match, starting from an offset point

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import copy

from pycbc import conversions, psd
from pycbc.filter import match

from simple_pe.param_est import metric, pe
from pesummary.utils.samples_dict import SamplesDict
from pesummary.gw.conversions import convert

In [2]:
import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)

## GW190412-like signal

## Read in PSDs, etc

In [3]:
ifos = ['H1', 'L1', 'V1']

psds = {'H1': 'aLIGOMidHighSensitivityP1200087',
        'L1': 'aLIGOMidHighSensitivityP1200087',
        'V1': 'AdVMidHighSensitivityP1200087',
        'f_low': 20.,
        'f_high': 8192,
        'length': 32
        }

psds['delta_f'] = 1. / psds['length']

approximant = 'IMRPhenomXPHM'

In [4]:
pycbc_psd = {}
for ifo in ifos:
    pycbc_psd[ifo] = psd.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                psds['f_low'])

pycbc_psd['harm'] = 3. / sum([1. / pycbc_psd[ifo] for ifo in ifos])

/home/stephen.fairhurst/.conda/envs/eccentric/lib/python3.9/site-packages/pycbc/types/array.py:390: RuntimeWarning: divide by zero encountered in true_divide
  return self._data.__rtruediv__(other)


## Parameters

In [5]:
m1 = 30.
m2 = 10.
mc = conversions.mchirp_from_mass1_mass2(m1, m2)
eta = conversions.eta_from_mass1_mass2(m1, m2)
s1z = 0.3
s2z = 0.
chi_eff = conversions.chi_eff(m1, m2, s1z, s2z)
dist = 1.
snr = 18

In [6]:
params = {'mass_1': m1,
          'mass_2': m2,
          'spin_1z': s1z,
          'spin_2z': s2z,
          'distance': dist
        }
params = pe.SimplePESamples(params)

## Make a waveform

In [7]:
h0 = metric.make_waveform(params, psds['delta_f'], psds['f_low'], 
                         len(pycbc_psd['harm']), approximant)


Offset the starting point

In [8]:
start = copy.deepcopy(params)
start['spin_1z'] += 0.1
start['spin_2z'] += 0.1

Now try to find the peak

In [9]:
dx_directions = ['spin_1z', 'spin_2z']
mismatch = 0.03

In [10]:
start = copy.deepcopy(params)
start['spin_1z'] += 0.1
start['spin_2z'] += 0.1

In [11]:
x_peak, match = metric.find_best_match(h0, start, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, method='scipy')

In [12]:
print("mismatch: %.4g" % (1 - match) )
print("at parameters")
print(x_peak)

mismatch: 3.109e-15
at parameters
{'spin_1z': 0.2999997047374733, 'spin_2z': 1.273701573855085e-06}


### Offset masses

In [13]:
params = {'chirp_mass': mc,
          'symmetric_mass_ratio': eta,
          'chi_eff': chi_eff,
          'distance': dist
        }

params = pe.SimplePESamples(params)

In [14]:
h0 = metric.make_waveform(params, psds['delta_f'], psds['f_low'], 
                         len(pycbc_psd['harm']), approximant)

In [15]:
start = copy.deepcopy(params)
start['chirp_mass'] += 0.1
start['chi_eff'] += 0.1

In [17]:
dx_directions = ['chirp_mass', 'symmetric_mass_ratio', 'chi_eff']
mismatch = 0.02

In [19]:
x_peak, match = metric.find_best_match(h0, start, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, method='scipy')

In [20]:
print("mismatch: %.4g" % (1 - match) )
print("at parameters")
print(x_peak)

mismatch: 1.592e-11
at parameters
{'chirp_mass': 14.650772466618802, 'symmetric_mass_ratio': 0.1875000896897749, 'chi_eff': 0.22499761025848566}
